In [1]:
from pyspark.mllib.recommendation import *
import random
from operator import *

In [2]:
userItemFile = sc.textFile("tianchi_fresh_comp_train_user.txt")
itemFile = sc.textFile("tianchi_fresh_comp_train_item.txt")

In [3]:
userItemFile.take(5)

[u'10001082\t285259775\t1', u'10001082\t4368907\t1', u'10001082\t4368907\t1', u'10001082\t53616768\t1', u'10001082\t151466952\t1']

In [4]:
itemFile.take(5)

[u'100002303\t3368', u'100003592\t7995', u'100006838\t12630', u'100008089\t7791', u'100012750\t9614']

In [3]:
def parser1(s):
    temp = s.split("\t")
    return (int(temp[0]),int(temp[1]),int(temp[2]))

def parser2(s):
    temp = s.split("\t")
    return (int(temp[0]),int(temp[0]))

In [4]:
userItem = userItemFile.map(parser1).map(lambda x: ((x[0],x[1]),x[2])).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0],x[0][1],x[1]))
item = itemFile.map(parser2).map(lambda x: (x[0],x[1]))

In [7]:
userItem.take(5)

[(10238478, 83906594, 1), (107386463, 192914757, 2), (104448961, 297013625, 3), (101982646, 173753750, 1), (111435559, 334399291, 1)]

In [8]:
userItem.count()

397276

In [9]:
item.take(5)

[(100002303, 100002303), (100003592, 100003592), (100006838, 100006838), (100008089, 100008089), (100012750, 100012750)]

In [10]:
item.count()

620918

In [5]:
data, b = userItem.randomSplit([1,7], 13)

In [6]:
trainData,validationData,testData = data.randomSplit([6,2,2], 13)
# print(trainData.take(3))
# print(validationData.take(3))
# print(testData.take(3))
print(trainData.count())
print(validationData.count())
print(testData.count())
trainData.cache()
validationData.cache()
testData.cache()

29691
9846
10050


PythonRDD[13] at RDD at PythonRDD.scala:48

In [13]:
def modelEval(bestModel,validation):
    import operator
    userIds = validation.map(lambda x: x[0]).distinct().collect()
    itemIds = list(item.collectAsMap().keys())
    temp = trainData.map(lambda x: (x[0], x[1])).filter(lambda x: x[0] in userIds).groupByKey().map(lambda x: (x[0], list(x[1])))
    print '1'
    valiSet = temp.flatMap(lambda x: [(x[0],a) for a in itemIds if a not in x[1]])
    print '2'
    predictions = bestModel.predictAll(valiSet).map(lambda x: (x[0], x[1], x[2])).groupBy(lambda x:x[0]).map(lambda x:(x[0],sorted(list(x[1]),key=operator.itemgetter(2),reverse=True)))
    print '2.5'
    predictions = predictions.map(lambda x:(x[0],list(elmt[1] for elmt in x[1])))
    print '3'
    cmprSet = validation.groupBy(lambda x:x[0]).map(lambda x:(x[0],list(x[1]))).map(lambda x:(x[0],list(elmt[1] for elmt in x[1])))
    print '4'
    p = predictions.collect()
    cdic =  cmprSet.collectAsMap()  
    scores = []
    print '5'
    for i in range(0,len(p)):
        user = p[i][0]
        plist = p[i][1]
        clist = cdic[user]
        plen = len(plist)
        clen = len(clist)
        count = 0
        score = 0
        if plen >= clen:
            for j in range(0,clen):
                if plist[j] in clist:
                    count += 1
        score = float(float(count)/clen)
        scores.append(score)
    print '6'
    return sum(scores)/len(scores)

In [14]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
score = 0
best_rank = 2
for elmt in range(2,11,2):
    print elmt
    bestModel = ALS.trainImplicit(trainData, rank=elmt)
    best_model_score = modelEval(bestModel,validationData)
    if model_score > score:
        score = best_model_score
        best_rank = elmt

print('The best model score for rank ' + str(best_rank) + ' is '+ str(best_model_score))

2
1
2
2.5
3
4


Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/tmp/kernel-PySpark-1509ca97-63bb-4992-894a-a82ddae122c7/pyspark_runner.py", line 189, in <module>
    eval(compiled_code)
  File "<string>", line 7, in <module>
  File "<string>", line 15, in modelEval
  File "/home/msbd5003/spark/python/pyspark/rdd.py", line 809, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/home/msbd5003/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1131, in __call__
    answer = self.gateway_client.send_command(command)
  File "/home/msbd5003/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/home/msbd5003/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1028, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python2.7/socket.py", line 451, in readline


In [7]:
bestModel = ALS.trainImplicit(trainData, rank=6)

In [8]:
temp = bestModel.recommendProducts(104448961, 5)

In [9]:
lists = []
for i in range(0,len(temp)):
    lists.append(temp[i][1])
    print("Item "+str(i+1)+": "+str(temp[i][1]))

Item 1: 237160017
Item 2: 342449091
Item 3: 335920490
Item 4: 140342917
Item 5: 44796654


In [10]:
import math

In [12]:
def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    """
    Grid Search Function to select the best model based on RMSE of hold-out data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # train ALS model
            model = ALS.train(
                ratings=train_data,    # (userID, productID, rating) tuple
                iterations=num_iters,
                rank=rank,
                lambda_=reg,           # regularization param
                seed=99)
            # make prediction
            valid_data = validation_data.map(lambda p: (p[0], p[1]))
            predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
            # get the rating result
            ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
            # get the RMSE
            MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
            error = math.sqrt(MSE)
            print('{} latent factors: validation RMSE is {}'.format(rank, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors'.format(best_rank))
    return best_model

In [13]:
# hyper-param config
num_iterations = 5
ranks = [2, 4, 6, 8, 10]
reg_params = [0.01]

In [14]:
final_model = train_ALS(trainData, validationData, num_iterations, reg_params, ranks)

2 latent factors: validation RMSE is 5.64686708211
4 latent factors: validation RMSE is 5.64575302802
6 latent factors: validation RMSE is 4.91813720378
8 latent factors: validation RMSE is 4.94902796925
10 latent factors: validation RMSE is 4.91260713016

The best model has 10 latent factors
